<a href="https://colab.research.google.com/github/LUMII-AILab/NLP_Course/blob/main/notebooks/MSP/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lielo valodas modeļu novērtēšana

In [ ]:
# Uzninstalējam OpenAi plaformu LVM darbināšanai
!pip install openai

In [ ]:
# Izmantosim 3 modeļus: bāzes modeļus TildeOpen un Gemma-2-27b un pielāgoto modeli gemma-2-27b-it
models = {
    "google/gemma-2-27b-it": "http://gauja.ailab.lv:30051/v1",
    "google/gemma-2-27b": "http://gauja.ailab.lv:30052/v1",
    "TildeAI/TildeOpen-30b": "http://gauja.ailab.lv:30053/v1"
}

##  Vienkāršota LVM darbināšana

In [ ]:
# Vienkāršu uzvedņu piemēri bāzes (my_prompt) un pielāgotajam (my_prompt_instruct) modelim
my_prompt_instruct="Slavens Latvijas komponists ir Raimonds \n A: Pauls \n B: Zirdziņš \n C: Bērziņš \n D: Lakstīgala \n Answer the above question by replying with 'a', 'b', 'c' or 'd', and nothing else. "
my_prompt="Slavens Latvijas komponists ir Raimonds "
prefix= "The following are multiple choice questions (with answers)."
# instruction_prompt="Question: {text} \n Options: \na. {option_a} \n b. {option_b} \n c. {option_c} \n d. {option_d} \n Answer the above question by replying with 'a', 'b', 'c' or 'd', and nothing else."

In [ ]:
# Bāzes modeļu darbināšana

from openai import OpenAI
# model = "TildeAI/TildeOpen-30b"
model = "google/gemma-2-27b"
client = OpenAI(
    base_url=models[model],
    api_key="any_key",
)
response=client.completions.create(
 #   prompt="Latvijā slavens komponists ir Raimonds",
    prompt=my_prompt,
    max_tokens=10,
    temperature=1,
    model=model
)
print(response.choices[0].text)

Ļaizāns un leks Montero.


In [ ]:
# Pielāgota modeļa darbināšana
# Completion komanda domāta teksta turpināšanai
from openai import OpenAI
model = "google/gemma-2-27b-it"
client = OpenAI(
    base_url=models[model],
    api_key="any_key"
)
response=client.completions.create(
    prompt=my_prompt,
    max_tokens=10,
    temperature=1,
    model=model
)
print(response.choices[0].text)


Pauls, kas izveidoja popul


In [ ]:
# Pielāgota modeļa darbināšana
# chat komanda ļauj labāk iegūt atbildi uz jautājumu
from openai import OpenAI
model = "google/gemma-2-27b-it"
client = OpenAI(
    base_url=models[model],
    api_key="any_key",
)
# Response=client.completions.create
response=client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": prefix + my_prompt_instruct,
        },
    ],
)
print(response.choices[0].message.content)

a




## Novērtēšana ar datu kopām


### LV-Exams datu kopa

In [ ]:
import re
import random
import requests
from openai import OpenAI

model = "google/gemma-2-27b-it"
client = OpenAI(
    base_url=models[model],
    api_key="any_key",
)

questions = requests.get("https://raw.githubusercontent.com/LUMII-AILab/NLP_Course/refs/heads/main/notebooks/resources/lv_exams.json").json()
random.seed(42)
random.shuffle(questions)

def format_question(question):
  text = question["question"] + "\n"
  for k, v in question["options"].items():
    text += f"{k}: {v}\n"

  return text

correct = 0
total = 0
for q in questions[:10]:
  prompt = format_question(q)
  prompt += "Atbildi formātā 'Atbilde ir: X', kur X ir pareizās atbildes burts.\n"
  # prompt += "Atbildiet uz jautājumu, atbildot ar 'A' vai 'B' vai 'C' vai 'D', un nekas cits.\n"


  response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": prompt
                    }
                ]
            }
        ],
        temperature=1,
        max_tokens=10,
    )
  response_text = response.choices[0].message.content.strip()
  m = re.search(r"Atbilde ir[:*\s]*([A-Z])", response_text, re.I | re.U)
  model_answer = m.group(1) if m is not None else "None"
  correct += model_answer == q['answer']
  # correct += response_text == q['answer']
  total += 1
  print(prompt)
  print("-------------")
  # print(f"Expected {q['answer']}, model guessed {model_answer}, full response: {response_text}")
  print(f"Expected {q['answer']}, full response: {response_text}")
  print("\n========\n")

print("Accuracy:", correct/total)




Kuras iezīmes raksturīgas cunami?
A: Tiem raksturīgi īsi viļņi un liela amplitūda. Parasti pārvietojas ar ātrumu 30 km/h. Viļņu augstums samazinās, sasniedzot piekrasti.
B: Tiem raksturīgi gari viļņi un maza amplitūda. Ātrums dažreiz sasniedz 700 km/h. Jūras līčos seklūdens zonā strauji palielinās viļņu augstums.
C: Tiem raksturīgi īsi viļņi, kuri veido vairākas viļņu sērijas. Viļņi ļoti strauji sasniedz sauszemi, kuras tuvumā augstums dažreiz pārsniedz 20 metrus.
D: Stipra vēja iekustināti lieli viļņi, kuri pārvietojoties attīstās par ļoti spēcīgiem viļņiem. Jūras līčos seklūdens zonā veidojas gigantiska izmēra viļņi.
Atbildi formātā 'Atbilde ir: X', kur X ir pareizās atbildes burts.

-------------
Expected B, full response: Atbilde ir: B


Sadedzinot divus litrus gāzveida ogļūdeņraža skābekļa pārākumā, izveidojās astoņi litri ogļskābās gāzes CO2 un desmit litru ūdens tvaika H2O. Visu vielu tilpumi mērīti vienādos apstākļos 100 °C un atmosfēras spiedienā. Kāda ir ogļūdeņraža molekulfo

### BeleBele datu kopa

In [ ]:
import json
import re
import random
import datasets
from openai import OpenAI

model = "google/gemma-2-27b-it"
client = OpenAI(
    base_url=models[model],
    api_key="any_key",
)

def format_question(question):
  text = question["flores_passage"] + "\n"
  text += question["question"] + "\n"
  letters = ['A', 'B', 'C', 'D']
  for i in range(4):
    text += f"{letters[i]}: {question['mc_answer'+str(i+1)]}\n"
    if str(i+1) == question['correct_answer_num']:
      question['answer'] = letters[i]

  return text

dataset = datasets.load_dataset('facebook/belebele', 'lvs_Latn')
correct = 0
total = 0
for q in dataset['test'].select(range(10)):
  prompt = format_question(q)
  prompt += "Atbildi formātā 'Atbilde ir: X', kur X ir pareizās atbildes burts.\n"


  response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": prompt
                    }
                ]
            }
        ],
        temperature=1,
        max_tokens=10,
    )
  response_text = response.choices[0].message.content
  m = re.search(r"Atbilde ir[:*\s]*([A-Z])", response_text, re.I | re.U)
  model_answer = m.group(1) if m is not None else "None"
  correct += model_answer == q['answer']
  total += 1
  print(prompt)
  print("-------------")
  print(f"Expected {q['answer']}, model guessed {model_answer}, full response: {response_text}")
  print("\n========\n")



print("Accuracy:", correct/total)




Gādājiet, lai jūsu roka būtu cik vien iespējams atbrīvota, bet tai pašā laikā visas notis spēlētu pareizi; mēģiniet arī neveikt liekas kustības ar pirkstiem. Tādā veidā jūs nogurdināsiet sevi pēc iespējas mazāk. Atcerieties, ka papildu skaļumam, piemēram, uz klavierēm, nav nepieciešams spiest taustiņus ar lielu spēku. Spēlējot akordeonu, skaļuma palielināšanai plēšas tiek vilktas ar lielāku spiedienu vai ātrumu.
Saskaņā ar fragmentu — kas nav uzskatāms par pareizu padomu sekmīgai akordeona spēlēšanai?
A: Lai panāktu papildu skaļumu, palieliniet spēku, ar kādu spiežat taustiņus
B: Līdz minimumam samaziniet liekas kustības, lai sevi nenogurdinātu
C: Domājiet līdzi, kādas notis spēlējat, un vienlaikus saglabājiet atbrīvotu roku
D: Lai panāktu papildu skaļumu, palieliniet ātrumu, ar kādu darbināt plēšas
Atbildi formātā 'Atbilde ir: X', kur X ir pareizās atbildes burts.

-------------
Expected A, model guessed A, full response: Atbilde ir: A 




Gādājiet, lai jūsu roka būtu cik vien iespēj